## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-07-06-57-50 +0000,nypost,Chicago special education teacher vanishes wit...,https://nypost.com/2026/01/07/us-news/chicago-...
1,2026-01-07-06-41-19 +0000,wapo,Trump administration freezes child care funds ...,https://www.washingtonpost.com/nation/2026/01/...
2,2026-01-07-06-35-11 +0000,nyt,European Allies Agree to Key Security Provisio...,https://www.nytimes.com/2026/01/06/world/europ...
3,2026-01-07-06-34-38 +0000,bbc,Opposition leader Machado says she should be i...,https://www.bbc.com/news/articles/c0er4qd3115o...
4,2026-01-07-06-28-14 +0000,nypost,Tennessee university reinstates professor fire...,https://nypost.com/2026/01/07/us-news/austin-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,55
35,venezuela,49
47,maduro,21
83,oil,21
247,new,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...,167
76,2026-01-07-00-24-38 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,157
69,2026-01-07-00-32-44 +0000,nypost,Venezuela selling up to 50 million barrels of ...,https://nypost.com/2026/01/06/us-news/venezuel...,148
23,2026-01-07-04-13-00 +0000,wsj,Venezuela to Give U.S. Up to 50 Million Barrel...,https://www.wsj.com/politics/policy/venezuela-...,148
32,2026-01-07-03-19-29 +0000,nyt,Trump Claims Venezuela Will Send Oil to U.S.,https://www.nytimes.com/2026/01/06/business/ve...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,167,2026-01-06-22-21-00 +0000,wsj,"The White House said the real crime on Jan. 6,...",https://www.wsj.com/politics/policy/white-hous...
76,102,2026-01-07-00-24-38 +0000,bbc,Trump says Venezuela will be 'turning over' up...,https://www.bbc.com/news/articles/c4grxzxjjd8o...
220,61,2026-01-06-17-47-55 +0000,nypost,AI deepfakes of Nicolás Maduro flood social me...,https://nypost.com/2026/01/06/business/ai-deep...
154,57,2026-01-06-21-51-00 +0000,wsj,Secretary of State Rubio told lawmakers that r...,https://www.wsj.com/politics/national-security...
105,47,2026-01-06-23-17-54 +0000,latimes,New California tool can stop brokers from sell...,https://www.latimes.com/california/story/2026-...
285,46,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
240,41,2026-01-06-16-16-08 +0000,nypost,Security clash with protesters in Iran’s Grand...,https://nypost.com/2026/01/06/world-news/secur...
174,40,2026-01-06-20-45-00 +0000,wsj,U.K. and France Agree to Set Up Military Hubs ...,https://www.wsj.com/world/europe/u-k-and-franc...
143,34,2026-01-06-22-02-12 +0000,nypost,Tim Walz melts down after dropping reelection ...,https://nypost.com/2026/01/06/us-news/tim-walz...
300,32,2026-01-06-10-30-23 +0000,bbc,Nvidia unveils self-driving car tech as it see...,https://www.bbc.com/news/articles/c0jv1vd571wo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
